## Segmenting and Clustering Neighborhoods in Toronto - Part 1

### Importing relevant files, particularly beautifulsoup and lxlm

In [1]:
import numpy as np    # library to handle data in a vectorized manner
import pandas as pd   # library for data analsysis
import requests
!easy_install beautifulsoup4   # installing beautifulsoup
!easy_install lxml             # installing parser

Searching for beautifulsoup4
Best match: beautifulsoup4 4.9.3
Processing beautifulsoup4-4.9.3-py3.6.egg
beautifulsoup4 4.9.3 is already the active version in easy-install.pth

Using /home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/beautifulsoup4-4.9.3-py3.6.egg
Processing dependencies for beautifulsoup4
Finished processing dependencies for beautifulsoup4
Searching for lxml
Best match: lxml 4.6.1
Processing lxml-4.6.1-py3.6-linux-x86_64.egg
lxml 4.6.1 is already the active version in easy-install.pth

Using /home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/lxml-4.6.1-py3.6-linux-x86_64.egg
Processing dependencies for lxml
Finished processing dependencies for lxml


### Uploading and organizing the table of data from Wikipedia

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url)

tdot_loc = soup.find('table',{'class':'wikitable sortable'})
links = tdot_loc.findAll('td')

roughpst = []
bor = []
neigh = []

pst_count = 0
bor_count = 1
neigh_count = 2

for j in links:
    if pst_count <= len(links)-3:
        roughpst.append(links[pst_count])
        pst_count = pst_count + 3
        
    if bor_count <= len(links)-2:
        bor.append(links[bor_count])
        bor_count = bor_count + 3
        
    if neigh_count <= len(links)-1:
        neigh.append(links[neigh_count])
        neigh_count = neigh_count + 3

roughpst = pd.DataFrame(roughpst)
bor= pd.DataFrame(bor)
neigh = pd.DataFrame(neigh)

### Removing \n in each string of the table

In [3]:
post_1 = []
post_2 = []
post_3 = []

for k in range(0,180,1):
    post_1.append(roughpst[0][k].replace('\n',''))
    post_2.append(bor[0][k].replace('\n',''))
    post_3.append(neigh[0][k].replace('\n',''))
    if k == 180:
        post_1.append(roughpst[0][k-1].replace('\n',''))
        post_2.append(bor[0][k-1].replace('\n',''))
        post_3.append(neigh[0][k-1].replace('\n',''))

data1 = {'Postal Code': post_1, 'Borough': post_2, 'Neighbourhood': post_3}

tdot_table = pd.DataFrame(data1)

tdot_table

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


### Removing rows that postal codes are not assigned to

In [4]:
postal = []
BOR = []
NEI = []

for NA in range(0,len(tdot_table),1):
    if tdot_table['Borough'][NA] != 'Not assigned' :    # if borough is not assigned THEN the neighbourhood column is also not assigned 
        postal.append(tdot_table['Postal Code'][NA])
        BOR.append(tdot_table['Borough'][NA])
        NEI.append(tdot_table['Neighbourhood'][NA])

data2 = {'Postal Code': postal, 'Borough': BOR, 'Neighbourhood': NEI}

real_tdot_table = pd.DataFrame(data2)

### Final Table of Toronto Postal Codes

In [5]:
real_tdot_table

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Using the .shape method to print the number of rows of my final dataframe

In [6]:
real_tdot_table.shape

(103, 3)